In [ ]:
import pandas as pd

In [ ]:
mart = pd.read_csv('mart_export.txt', sep='\t')
mart_hom = mart[mart['Human homology type']=='ortholog_one2one']
mart_hom = mart_hom[['Gene stable ID', 'Human gene stable ID']]
mart_hom = mart_hom.rename(columns={'Human gene stable ID':'ENSEMBL', 'Gene stable ID':'ENSEMBL_mice'})
mart_hom_clean = mart_hom.drop_duplicates(subset="ENSEMBL")
sen = pd.read_csv('senescence_data.csv')
sen_mouse = pd.merge(sen, mart_hom_clean, on='ENSEMBL')
sen_mouse.to_csv("senescence_mouse.csv")
auto = pd.read_csv('autophagy_signatures.csv')
auto_mouse = pd.merge(auto, mart_hom_clean, on='ENSEMBL')
auto_mouse.to_csv("autophagy_mouse.csv")
proteo = pd.read_csv('proteostasis_signatures.csv')
proteo_mouse = pd.merge(proteo, mart_hom_clean, on='ENSEMBL')
proteo_mouse.to_csv("proteostasis_mouse.csv")

In [ ]:
auto_mouse = auto_mouse.rename(columns = {"Process" : "Subprocess"})
auto_mouse.to_csv("autophagy_mouse.csv")

In [ ]:
metadata_df = pd.read_csv('metadata.csv', index_col=0)
vst_df = pd.read_csv('vst_normalized_counts.csv', index_col=0)

## Spearman by gene; overall

In [ ]:
import pandas as pd
from scipy.stats import spearmanr, zscore

def spearman_gene_bulk(hallmark_df_path, vst_df, metadata_df, name_of_hallmark, sp_border=0.4):
    vst_df = vst_df.apply(pd.to_numeric, errors='coerce')
    vst_df.index = vst_df.index.astype(str)

    hallmark_df = pd.read_csv(hallmark_df_path)
    hallmark_df.columns = hallmark_df.columns.str.strip()
    sub_hallmarks = hallmark_df['Subprocess'].dropna().unique()

    spearman_df = pd.DataFrame(columns=['ENSEMBL_mice', 'Spearman', 'p-value', 'Subprocess', 'Hallmark'])

    vst_gene_set = set(vst_df.index)

    for proc in sub_hallmarks:
        subproc_genes = hallmark_df[hallmark_df['Subprocess'] == proc]['ENSEMBL_mice']
        subproc_genes = [str(gene) for gene in subproc_genes if str(gene) in vst_gene_set]

        if not subproc_genes:
            continue

        sub_df = vst_df.loc[subproc_genes]
        zscored_array = zscore(sub_df, axis=1, nan_policy='omit')
        zscored_df = pd.DataFrame(zscored_array, index=sub_df.index, columns=sub_df.columns).transpose()
        zscored_df = zscored_df.clip(lower=-5, upper=5)

        zscored_df = zscored_df.loc[metadata_df.index]
        zscored_df['Day'] = metadata_df['Day'].astype('int')

        zscored_df.index.name = 'Sample'
        zscored_long = zscored_df.reset_index().melt(
            id_vars=['Sample', 'Day'],
            var_name='Gene',
            value_name='Zscore'
        )
        zscored_long.rename(columns={'index': 'Sample'}, inplace=True)

        spearman_results = []
        for gene, group in zscored_long.groupby('Gene'):
            rho, pval = spearmanr(group['Day'], group['Zscore'])
            spearman_results.append((gene, rho, pval, proc, name_of_hallmark))

        spearman_results_df = pd.DataFrame(spearman_results,
                                           columns=['ENSEMBL_mice', 'Spearman', 'p-value', 'Subprocess', 'Hallmark'])
        spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)

    spearman_df_signif = spearman_df[abs(spearman_df['Spearman']) >= sp_border]
    print(f'Number of genes with abs(Spearman) >= {sp_border}: {len(spearman_df_signif)}')

    return spearman_df_signif

In [ ]:
spearman_gene_bulk("senescence_mouse.csv", vst_df, metadata_df, "Senescence", sp_border=0.3)

## Skin

In [ ]:
skin_samples = metadata_df[metadata_df['Tissue'] == 'Skin'].index
vst_skin_df = vst_df[skin_samples]
metadata_skin_df = metadata_df.loc[skin_samples]

In [ ]:
sen_table = pd.read_csv("senescence_mouse.csv", index_col=0)
auto_table = pd.read_csv("autophagy_mouse.csv", index_col=0)
proteo_table = pd.read_csv("proteostasis_mouse.csv", index_col=0)
genom_table = pd.read_csv("genomic_signatures_new.csv", index_col=0)

In [ ]:
skin_g1 = spearman_gene_bulk("senescence_mouse.csv", vst_skin_df, metadata_skin_df, "Senescence", sp_border=0.2)
skin_g1 = skin_g1.merge(sen_table[['ENSEMBL_mice', 'Gene']], on='ENSEMBL_mice', how='left')
skin_g1

<ipython-input-40-95dedf07e46d>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.2: 65


,ENSEMBL_mice,Spearman,p-value,Subprocess,Hallmark,Gene
0,ENSMUSG00000000489,0.315405,0.005515,SASP,Senescence,PDGFB
1,ENSMUSG00000009281,0.252599,0.027704,SASP,Senescence,RARRES2
2,ENSMUSG00000020676,0.250758,0.028899,SASP,Senescence,CCL11
3,ENSMUSG00000022636,-0.297663,0.009016,SASP,Senescence,ALCAM
4,ENSMUSG00000023078,0.291204,0.010706,SASP,Senescence,CXCL13
...,...,...,...,...,...,...
60,ENSMUSG00000042677,0.210421,0.068077,Other,Senescence,ZC3H12A
61,ENSMUSG00000044317,0.214297,0.063042,Other,Senescence,GPR4
62,ENSMUSG00000049892,0.276430,0.015639,Other,Senescence,RASD1
63,ENSMUSG00000087006,0.234816,0.041172,Other,Senescence,C11orf96


In [ ]:
genom_table = genom_table.reset_index()

In [ ]:
skin_g2 = spearman_gene_bulk("autophagy_mouse.csv", vst_skin_df, metadata_skin_df, "Autophagy", sp_border=0.2)
skin_g2 = skin_g2.merge(auto_table[['ENSEMBL_mice', 'Gene']], on='ENSEMBL_mice', how='left')
skin_g3 = spearman_gene_bulk("proteostasis_mouse.csv", vst_skin_df, metadata_skin_df, "Proteostasis", sp_border=0.2)
skin_g3 = skin_g3.merge(proteo_table[['ENSEMBL_mice', 'Gene']], on='ENSEMBL_mice', how='left')
skin_g4 = spearman_gene_bulk("genomic_signatures_new.csv", vst_skin_df, metadata_skin_df, "Genomic Instability", sp_border=0.2)
skin_g4 = skin_g4.merge(genom_table[['ENSEMBL_mice', 'Gene']], on='ENSEMBL_mice', how='left')
spearman_gene_subp = pd.concat([skin_g1, skin_g2, skin_g3, skin_g4], ignore_index=True)
spearman_gene_subp

<ipython-input-40-95dedf07e46d>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.2: 171


<ipython-input-40-95dedf07e46d>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.2: 580


<ipython-input-40-95dedf07e46d>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.2: 25


,ENSEMBL_mice,Spearman,p-value,Subprocess,Hallmark,Gene
0,ENSMUSG00000000489,0.315405,0.005515,SASP,Senescence,PDGFB
1,ENSMUSG00000009281,0.252599,0.027704,SASP,Senescence,RARRES2
2,ENSMUSG00000020676,0.250758,0.028899,SASP,Senescence,CCL11
3,ENSMUSG00000022636,-0.297663,0.009016,SASP,Senescence,ALCAM
4,ENSMUSG00000023078,0.291204,0.010706,SASP,Senescence,CXCL13
...,...,...,...,...,...,...
1113,ENSMUSG00000023104,-0.210765,0.067618,DNA Replication/Repair,Genomic Instability,RFC2
1114,ENSMUSG00000021287,0.262797,0.021819,Homologous Recombination,Genomic Instability,XRCC3
1115,ENSMUSG00000040174,0.329712,0.003631,Direct Reversal Repair,Genomic Instability,ALKBH3
1116,ENSMUSG00000054612,0.264240,0.021079,Direct Reversal Repair,Genomic Instability,MGMT


In [ ]:
spearman_gene_subp = spearman_gene_subp.drop(['ENSEMBL_mice'], axis=1)
spearman_gene_subp.rename(columns={'Gene':'Gene Symbol'}, inplace=True)
new_order = ['Hallmark', 'Subprocess', 'Gene Symbol', 'Spearman', 'p-value']
spearman_gene_subp = spearman_gene_subp[new_order]
spearman_gene_subp['Experiment'] = 'Ageing-Skin'
spearman_gene_subp['Species'] = 'Mouse'
spearman_gene_subp

,Hallmark,Subprocess,Gene Symbol,Spearman,p-value,Experiment,Species
0,Senescence,SASP,PDGFB,0.315405,0.005515,Ageing-Skin,Mouse
1,Senescence,SASP,RARRES2,0.252599,0.027704,Ageing-Skin,Mouse
2,Senescence,SASP,CCL11,0.250758,0.028899,Ageing-Skin,Mouse
3,Senescence,SASP,ALCAM,-0.297663,0.009016,Ageing-Skin,Mouse
4,Senescence,SASP,CXCL13,0.291204,0.010706,Ageing-Skin,Mouse
...,...,...,...,...,...,...,...
1113,Genomic Instability,DNA Replication/Repair,RFC2,-0.210765,0.067618,Ageing-Skin,Mouse
1114,Genomic Instability,Homologous Recombination,XRCC3,0.262797,0.021819,Ageing-Skin,Mouse
1115,Genomic Instability,Direct Reversal Repair,ALKBH3,0.329712,0.003631,Ageing-Skin,Mouse
1116,Genomic Instability,Direct Reversal Repair,MGMT,0.264240,0.021079,Ageing-Skin,Mouse


In [ ]:
spearman_gene_subp.to_csv("Ageing_skin_mouse_by_gene_spearman.csv")

In [ ]:
def count_spearman_conditions(df):
    conditions = {
        'n genes >= 0.2': (df['Spearman'] >= 0.2),
        'n genes <= 0.2': (df['Spearman'] <= 0.2),
        'n genes >= 0.4': (df['Spearman'] >= 0.4),
        'n genes <= -0.4': (df['Spearman'] <= -0.4)
    }

    count_dict = {}

    for condition_name, condition in conditions.items():
        count = df[condition].groupby(['Hallmark', 'Subprocess']).size()
        count_dict[condition_name] = count

    result_df = pd.DataFrame(count_dict).fillna(0).astype(int)
    result_df.reset_index(inplace=True)

    return result_df


In [ ]:
skin_gene_count = count_spearman_conditions(spearman_gene_subp)
skin_gene_count['Experiment'] = 'Ageing-Skin'
skin_gene_count['Species'] = 'Mouse'
skin_gene_count

,Hallmark,Subprocess,n genes >= 0.2,n genes <= 0.2,n genes >= 0.4,n genes <= -0.4,Experiment,Species
0,Autophagy,Autophagy core,34,9,1,0,Ageing-Skin,Mouse
1,Autophagy,Autophagy downregulators,2,0,0,0,Ageing-Skin,Mouse
2,Autophagy,Autophagy upregulators,19,2,0,0,Ageing-Skin,Mouse
3,Autophagy,Docking and fusion,2,0,0,0,Ageing-Skin,Mouse
4,Autophagy,Lysosome,30,4,0,0,Ageing-Skin,Mouse
5,Autophagy,Lysosome-related genes,4,3,0,0,Ageing-Skin,Mouse
6,Autophagy,Mitophagy,38,2,0,0,Ageing-Skin,Mouse
7,Autophagy,Other,17,4,0,0,Ageing-Skin,Mouse
8,Autophagy,mTOR and upstream pathways,31,4,0,0,Ageing-Skin,Mouse
9,Genomic Instability,Auxiliary Processes,1,0,0,0,Ageing-Skin,Mouse


In [ ]:
skin_gene_count.to_csv("Ageing_skin_mouse_gene_count_spearman.csv")

In [ ]:
skin_proteo = spearman_gene_bulk("proteostasis_mouse.csv", vst_skin_df, metadata_skin_df, "Proteostasis", sp_border=0.3)

<ipython-input-42-6f67603cbeb9>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.3: 89


In [ ]:
skin_proteo['Signature'] = 'Proteostasis'
skin_proteo

,Gene,Spearman_rho,p_value,subprocess,hallmark,Signature
75,ENSMUSG00000025092,0.306527,7.078758e-03,Cytonuclear proteostasis,Proteostasis,Proteostasis
127,ENSMUSG00000033209,0.310142,6.400514e-03,Cytonuclear proteostasis,Proteostasis,Proteostasis
138,ENSMUSG00000035992,0.338384,2.791543e-03,Cytonuclear proteostasis,Proteostasis,Proteostasis
250,ENSMUSG00000019732,0.301071,8.222304e-03,ER proteostasis,Proteostasis,Proteostasis
362,ENSMUSG00000032295,0.351605,1.842494e-03,ER proteostasis,Proteostasis,Proteostasis
...,...,...,...,...,...,...
2815,ENSMUSG00000049106,0.310540,6.329353e-03,Ubiquitin Proteasome System,Proteostasis,Proteostasis
2877,ENSMUSG00000055401,0.322827,4.451266e-03,Ubiquitin Proteasome System,Proteostasis,Proteostasis
2900,ENSMUSG00000059263,0.395253,4.666817e-07,Ubiquitin Proteasome System,Proteostasis,Proteostasis
2952,ENSMUSG00000071089,-0.346850,2.143838e-03,Ubiquitin Proteasome System,Proteostasis,Proteostasis


In [ ]:
skin_sen = spearman_gene_bulk("senescence_mouse.csv", vst_skin_df, metadata_skin_df, "Senescence", sp_border=0.3)

<ipython-input-42-6f67603cbeb9>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.3: 12


In [ ]:
skin_sen['Signature'] = 'Senescence'

In [ ]:
skin_auto = spearman_gene_bulk("autophagy_mouse.csv", vst_skin_df, metadata_skin_df, "Autophagy", sp_border=0.3)
skin_auto['Signature'] = 'Autophagy'

<ipython-input-42-6f67603cbeb9>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.3: 36


In [ ]:
skin_gen = spearman_gene_bulk("genomic_signatures_new.csv", vst_skin_df, metadata_skin_df, "Genomic", sp_border=0.2)
skin_gen['Signature'] = 'Genomic'

<ipython-input-42-6f67603cbeb9>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  spearman_df = pd.concat([spearman_df, spearman_results_df], ignore_index=True)


Number of genes with abs(Spearman) >= 0.2: 25


In [ ]:
spearman_skin = pd.concat([skin_proteo, skin_sen, skin_auto, skin_gen], ignore_index=True)
spearman_skin = spearman_skin.drop(columns='Signature')
spearman_skin

,Gene,Spearman_rho,p_value,subprocess,hallmark
0,ENSMUSG00000025092,0.306527,0.007079,Cytonuclear proteostasis,Proteostasis
1,ENSMUSG00000033209,0.310142,0.006401,Cytonuclear proteostasis,Proteostasis
2,ENSMUSG00000035992,0.338384,0.002792,Cytonuclear proteostasis,Proteostasis
3,ENSMUSG00000019732,0.301071,0.008222,ER proteostasis,Proteostasis
4,ENSMUSG00000032295,0.351605,0.001842,ER proteostasis,Proteostasis
...,...,...,...,...,...
157,ENSMUSG00000023104,-0.210765,0.067618,DNA Replication/Repair,Genomic
158,ENSMUSG00000021287,0.262797,0.021819,Homologous Recombination,Genomic
159,ENSMUSG00000040174,0.329712,0.003631,Direct Reversal Repair,Genomic
160,ENSMUSG00000054612,0.264240,0.021079,Direct Reversal Repair,Genomic


In [ ]:
spearman_skin.to_csv("Spearman_ageing_skin.csv")

### Overall Spearman by subprocess

In [ ]:
import pandas as pd
from scipy.stats import spearmanr, zscore

def spearman_subprocess_bulk_by_tissue(hallmark_df_path, vst_df, metadata_df, name_of_hallmark,
                                       sp_border=0.4, tissue=None):
    if tissue is not None:
        metadata_df = metadata_df[metadata_df['Tissue'] == tissue]
        vst_df = vst_df[metadata_df.index]

    vst_df = vst_df.apply(pd.to_numeric, errors='coerce')
    vst_df.index = vst_df.index.astype(str)

    hallmark_df = pd.read_csv(hallmark_df_path)
    hallmark_df.columns = hallmark_df.columns.str.strip()
    sub_hallmarks = hallmark_df['Subprocess'].dropna().unique()

    vst_gene_set = set(vst_df.index)
    results = []

    for proc in sub_hallmarks:
        subproc_genes = hallmark_df[hallmark_df['Subprocess'] == proc]['ENSEMBL_mice']
        subproc_genes = [str(gene) for gene in subproc_genes if str(gene) in vst_gene_set]

        if not subproc_genes:
            continue

        sub_df = vst_df.loc[subproc_genes]
        zscored_array = zscore(sub_df, axis=1, nan_policy='omit')
        zscored_df = pd.DataFrame(zscored_array, index=sub_df.index, columns=sub_df.columns).transpose()
        zscored_df = zscored_df.clip(lower=-5, upper=5)

        mean_zscore = zscored_df.mean(axis=1)
        mean_zscore.name = 'Mean_Zscore'

        mean_zscore = mean_zscore.loc[metadata_df.index]
        day_numeric = metadata_df['Day'].astype(int)

        rho, pval = spearmanr(day_numeric, mean_zscore)
        results.append((proc, rho, pval, name_of_hallmark, tissue if tissue else 'All'))

    results_df = pd.DataFrame(results, columns=['subprocess', 'Spearman_rho', 'p_value', 'hallmark', 'Tissue'])
    results_df_signif = results_df[abs(results_df['Spearman_rho']) >= sp_border]

    print(f'Tissue: {tissue if tissue else "All"} — Subprocesses with |rho| ≥ {sp_border}: {len(results_df_signif)}')
    return results_df


In [ ]:
metadata_df["Tissue"].unique()

array(['Muscle', 'Skin', 'Marrow', 'Spleen', 'Lung', 'Bone', 'SCAT',
       'MAT', 'WBC', 'BAT', 'Heart', 'Brain', 'Liver', 'Intestine',
       'Pancreas', 'GAT', 'Kidney', 'Blood', 'Colon'], dtype=object)

In [ ]:
spearman_subprocess_bulk_by_tissue("senescence_mouse.csv", vst_df, metadata_df, "Senescence", sp_border=0.2)

Tissue: All — Subprocesses with |rho| ≥ 0.2: 0


,subprocess,Spearman_rho,p_value,hallmark,Tissue
0,DNA damage response,0.041089,0.168070,Senescence,All
1,SASP,0.071374,0.016554,Senescence,All
2,SASP_Secreted,0.058133,0.051050,Senescence,All
3,Metabolic adaptations,0.033160,0.266012,Senescence,All
4,Cell cycle arrest,0.012401,0.677520,Senescence,All
5,Nuclear changes,0.017528,0.556655,Senescence,All
6,SASP_Transmembrane,0.061186,0.040005,Senescence,All
7,SASP_Intracellular,0.048071,0.106761,Senescence,All
8,Changes in morphology,0.016596,0.577831,Senescence,All
9,Upregulation of anti-apoptotic pathways,0.012862,0.666233,Senescence,All


In [ ]:
dfs  = pd.DataFrame()
for tissue in metadata_df["Tissue"].unique():
  df = spearman_subprocess_bulk_by_tissue("genomic_signatures_new.csv", vst_df, metadata_df, "Genomic", sp_border=0.2, tissue=tissue)
  dfs = pd.concat([dfs,df], ignore_index=True)

Tissue: Muscle — Subprocesses with |rho| ≥ 0.2: 0
Tissue: Skin — Subprocesses with |rho| ≥ 0.2: 3
Tissue: Marrow — Subprocesses with |rho| ≥ 0.2: 0
Tissue: Spleen — Subprocesses with |rho| ≥ 0.2: 9
Tissue: Lung — Subprocesses with |rho| ≥ 0.2: 5
Tissue: Bone — Subprocesses with |rho| ≥ 0.2: 3
Tissue: SCAT — Subprocesses with |rho| ≥ 0.2: 0
Tissue: MAT — Subprocesses with |rho| ≥ 0.2: 0
Tissue: WBC — Subprocesses with |rho| ≥ 0.2: 5
Tissue: BAT — Subprocesses with |rho| ≥ 0.2: 0
Tissue: Heart — Subprocesses with |rho| ≥ 0.2: 4
Tissue: Brain — Subprocesses with |rho| ≥ 0.2: 0
Tissue: Liver — Subprocesses with |rho| ≥ 0.2: 1
Tissue: Intestine — Subprocesses with |rho| ≥ 0.2: 0
Tissue: Pancreas — Subprocesses with |rho| ≥ 0.2: 4
Tissue: GAT — Subprocesses with |rho| ≥ 0.2: 0
Tissue: Kidney — Subprocesses with |rho| ≥ 0.2: 3
Tissue: Blood — Subprocesses with |rho| ≥ 0.2: 20
Tissue: Colon — Subprocesses with |rho| ≥ 0.2: 11


In [ ]:
dfs

,subprocess,Spearman_rho,p_value,hallmark,Tissue
0,Cell Cycle and Mitotic Regulation,0.049205,0.723840,Genomic,Muscle
1,Replication Stress Response,-0.126499,0.362037,Genomic,Muscle
2,Transcription-Replication Conflict,0.018662,0.893447,Genomic,Muscle
3,Chromatin and Nuclear Organization,-0.016593,0.905205,Genomic,Muscle
4,Nucleotide Excision Repair,-0.161908,0.242134,Genomic,Muscle
...,...,...,...,...,...
508,Homologous Recombination,-0.222802,0.042912,Genomic,Colon
509,Transcription-Coupled Repair,-0.260509,0.017379,Genomic,Colon
510,Direct Reversal Repair,0.239235,0.029390,Genomic,Colon
511,Single-Strand Break Repair,-0.039314,0.724187,Genomic,Colon


In [ ]:
dfs.to_csv("Genomic_spearman_by_process_tissues.csv")

In [ ]:
metadata_df['Day'].unique()

array([ 30,  60,  90, 180, 270, 360, 450, 540, 630, 720, 810, 840, 900])

In [ ]:
vst_df

,Sample_0,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8,Sample_9,...,Sample_1117,Sample_1118,Sample_1119,Sample_1120,Sample_1121,Sample_1122,Sample_1123,Sample_1124,Sample_1125,Sample_1126
ENSMUSG00000000001,11.083353,11.017505,9.832516,9.768736,9.314694,9.625790,9.632689,8.774494,10.391804,11.186102,...,10.281085,10.373040,8.741896,8.682300,8.702575,8.709753,8.662479,8.645045,8.671426,10.438476
ENSMUSG00000000028,5.415528,6.822946,6.496659,5.910625,6.530297,6.683910,5.355757,4.932569,5.858477,6.487810,...,7.348207,7.249768,5.398349,5.680971,5.570342,5.658417,5.419122,5.151995,5.622841,5.621299
ENSMUSG00000000031,1.119091,4.528415,5.477599,2.111818,9.775239,4.275795,0.753258,4.241372,9.354595,1.191889,...,7.728051,8.182386,0.854398,0.996189,1.095972,0.727391,0.417008,0.629829,0.851856,4.531619
ENSMUSG00000000037,3.133844,5.599212,3.109658,4.460909,3.205169,4.482453,4.538631,4.628055,4.973648,3.538058,...,5.811829,5.693150,4.604529,4.152517,4.655995,4.555029,4.500083,4.666443,4.583623,0.884487
ENSMUSG00000000049,14.087176,0.883066,4.064837,7.060277,6.441063,6.597773,4.600137,6.207034,4.477203,6.471414,...,3.191687,2.884393,2.287679,2.467483,2.076557,2.097767,2.484849,2.196506,2.138532,11.703175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000096900,1.119091,0.883066,1.105295,2.111818,1.257207,2.677955,1.754920,0.687573,1.053632,1.757526,...,0.395696,0.419757,0.245858,0.248116,0.297093,0.265176,0.417008,0.338103,0.340939,0.884487
ENSMUSG00000096908,1.119091,0.883066,1.105295,2.111818,1.257207,1.738473,0.753258,0.687573,2.240512,1.191889,...,0.395696,0.419757,0.245858,0.248116,0.297093,0.265176,0.417008,0.338103,0.340939,0.884487
ENSMUSG00000096910,7.007754,6.854938,6.345747,4.013365,7.036023,6.303400,6.244951,7.370215,6.742155,6.139097,...,6.251335,6.195071,6.943234,6.958787,6.862346,6.981101,6.958455,6.976828,7.110506,6.234780
ENSMUSG00000096914,1.119091,1.360494,2.563258,2.111818,1.257207,4.034376,4.796814,8.217517,2.240512,1.757526,...,0.395696,0.419757,6.921808,7.254714,7.375666,7.287373,7.000587,7.367941,6.615449,0.884487


In [ ]:
skin_2 = pd.read_csv('Autophagy_spearman_by_process_tissues.csv', index_col=0)
skin_4 = pd.read_csv('Genomic_spearman_by_process_tissues.csv', index_col=0)
skin_4 = skin_4.drop(['hallmark'], axis=1)
skin_4['hallmark'] = "Genomic Instability"
skin_3 = pd.read_csv('Proteostasis_spearman_by_process_tissues.csv', index_col=0)
skin_1 = pd.read_csv('Senescence_spearman_by_process_tissues.csv', index_col=0)
subp_skin = pd.concat([skin_1, skin_2, skin_3, skin_4], ignore_index=True)
subp_skin = subp_skin[subp_skin['Tissue']=='Skin'].drop(['Tissue'], axis=1)
subp_skin

,subprocess,Spearman_rho,p_value,hallmark
13,DNA damage response,0.043964,0.706097,Senescence
14,SASP,0.123400,0.288231,Senescence
15,SASP_Secreted,0.182276,0.115040,Senescence
16,Metabolic adaptations,0.191635,0.097246,Senescence
17,Cell cycle arrest,0.084754,0.466654,Senescence
18,Nuclear changes,-0.013317,0.909098,Senescence
19,SASP_Transmembrane,0.210985,0.067326,Senescence
20,SASP_Intracellular,0.221897,0.054044,Senescence
21,Changes in morphology,0.035760,0.759089,Senescence
22,Upregulation of anti-apoptotic pathways,-0.111897,0.335872,Senescence


In [ ]:
subp_skin['Experiment'] = "Ageing-Skin"
subp_skin['Species'] = "Mouse"
subp_skin.set_index(subp_skin.columns[3], inplace=True)
subp_skin

,subprocess,Spearman_rho,p_value,Experiment,Species
hallmark,,,,,
Senescence,DNA damage response,0.043964,0.706097,Ageing-Skin,Mouse
Senescence,SASP,0.123400,0.288231,Ageing-Skin,Mouse
Senescence,SASP_Secreted,0.182276,0.115040,Ageing-Skin,Mouse
Senescence,Metabolic adaptations,0.191635,0.097246,Ageing-Skin,Mouse
Senescence,Cell cycle arrest,0.084754,0.466654,Ageing-Skin,Mouse
Senescence,Nuclear changes,-0.013317,0.909098,Ageing-Skin,Mouse
Senescence,SASP_Transmembrane,0.210985,0.067326,Ageing-Skin,Mouse
Senescence,SASP_Intracellular,0.221897,0.054044,Ageing-Skin,Mouse
Senescence,Changes in morphology,0.035760,0.759089,Ageing-Skin,Mouse


In [ ]:
subp_skin = subp_skin.reset_index()

In [ ]:
subp_skin.rename(columns={'hallmark':'Hallmark','subprocess': 'Subprocess', 'Spearman_rho':'Spearman', 'p_value':'p-value'}, inplace=True)
subp_skin

,Hallmark,Subprocess,Spearman,p-value,Experiment,Species
0,Senescence,DNA damage response,0.043964,0.706097,Ageing-Skin,Mouse
1,Senescence,SASP,0.123400,0.288231,Ageing-Skin,Mouse
2,Senescence,SASP_Secreted,0.182276,0.115040,Ageing-Skin,Mouse
3,Senescence,Metabolic adaptations,0.191635,0.097246,Ageing-Skin,Mouse
4,Senescence,Cell cycle arrest,0.084754,0.466654,Ageing-Skin,Mouse
5,Senescence,Nuclear changes,-0.013317,0.909098,Ageing-Skin,Mouse
6,Senescence,SASP_Transmembrane,0.210985,0.067326,Ageing-Skin,Mouse
7,Senescence,SASP_Intracellular,0.221897,0.054044,Ageing-Skin,Mouse
8,Senescence,Changes in morphology,0.035760,0.759089,Ageing-Skin,Mouse
9,Senescence,Upregulation of anti-apoptotic pathways,-0.111897,0.335872,Ageing-Skin,Mouse


In [ ]:
subp_skin.to_csv("Ageing_skin_mouse_spearman_subprocess.csv")